# Lesson 4 : Mini Challenge

Use what you learned in Lesson 1 to claim testnet tokens from an airdrop contract

You will need to copy and paste the code from there into this notebook.

Make sure you create a new wallet using Metamask since we will be handling the private key in code

There is one new thing we are learning this week: how to interact with a contract. An example is below, you will need to integrate this with the code you write


### Interacting with a contract

In [2]:
from web3 import Web3
from datetime import datetime

import urllib
import json
import pandas as pd

In [ ]:
#GAS_AMOUNT = 500000
#MAX_GAS_PRICE = '40'
#PRIORITY_FEE = '2.5'

def send_mint_tx(wallet, private_key, num_wapes, contract, salesprice,
                GAS_AMOUNT, MAX_GAS_PRICE, PRIORITY_FEE):
    # Get price
    #===================================================
    saleStarted = contract.functions.saleStarted().call()
    #===================================================
    if not saleStarted:
        print(f"FATAL ERROR: sale has not started")        
        return (False, 0)   
    
    balance = w3.eth.getBalance(wallet)
    
    total_amt = num_wapes * salesprice
    
    #check sufficient balance
    total_amt_eth = w3.fromWei(total_amt, "ether")
    balance_eth = w3.fromWei(balance, "ether")
    if balance < total_amt:
        print(f"FATAL ERROR: insufficient balance: need {total_amt_eth} "
              f"but balance is only {balance_eth}")
        
        return (False, 0)
        
    nonce = w3.eth.getTransactionCount(wallet)

    # Call your function
    #===================================================
    call_function = contract.functions.purchaseDomainsPublicSale(num_wapes).buildTransaction(
        {"chainId": w3.eth.chain_id, 
         "from": wallet, 
         "nonce": nonce, 
         'gas': GAS_AMOUNT,
         'value' : total_amt,
         'maxFeePerGas': w3.toWei(MAX_GAS_PRICE, 'gwei'),
         'maxPriorityFeePerGas': w3.toWei(PRIORITY_FEE, 'gwei'),})
    #===================================================

    # Sign transaction
    signed_tx = w3.eth.account.sign_transaction(call_function, private_key=private_key)

    print(f"About to public mint on chainid {w3.eth.chain_id}, {num_wapes} wapes on wallet {wallet}")
    
    # Send transaction
    send_tx = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    return (True, send_tx)

### Load our API keys 

To avoid putting api keys on github where they can be read, you can create a .env file in the ww-blockchain-dev root folder.
Put your api keys in there, in the format:

<b>ETHERSCAN_API=...</b><br>
<b>ALCHEMY_API=...</b><br>
<b>MORALIS_API=...</b><br>

In [2]:
with open("..\..\.env", "r") as f:
    keys = f.read().splitlines()
    
for key in keys:
    if "ETHERSCAN_API" in key: etherscan_apikey = key.split('=')[1]
    if "ALCHEMY_API" in key: alchemy_apikey = key.split('=')[1]
    if "MORALIS_API" in key: moralis_key = key.split('=')[1]
    

### Connect to the Alchemy provider
To interact with the Ethereum blockchain, we either need to run a geth node on our machine, or we can use a third party provider like Alchemy or Infura

In [5]:
wss = f'wss://eth-mainnet.g.alchemy.com/v2/{alchemy_apikey}'
w3 = Web3(Web3.WebsocketProvider(wss))

print(w3.is_connected())

True


In [6]:
#We should be connected to chain_id 1, i.e. Ethereum mainnet
w3.eth.chain_id

1

### Token contract addresses 

Note that we have two addresses: a contract address which is actually a proxy, that delegates to the actual implementation contract where all the Solidity code is.

Delegation allows us to "upgrade" our smart contracts by repointing to a new implementation contract if the code changes (but note there are dangers with delegation that we need to be aware of)

In [7]:
# $WILD Contract Address
wild_contract = '0x2a3bff78b79a009976eea096a51a948a3dc00e34'
# Implementation Address
wild_implementation_contract = '0x67aac030b59d266e754b0b24af9cc77ec2534a37'

### Load the ABI (Application Binary Interface)

The ABI defines all the functions and function parameters of the smart contract. We need this info if we want to make calls to the contract

In [8]:
abi_endpoint = f"https://api.etherscan.io/api?module=contract&action=getabi&address={wild_implementation_contract}&apikey={etherscan_apikey}"

# Get the abi in json format from the etherscan API and decode it
with urllib.request.urlopen(abi_endpoint) as url:
    abi = json.loads(url.read().decode())

# Load this into the web3 contract object
checksum_addr = Web3.to_checksum_address(wild_contract)
contract = w3.eth.contract(address=checksum_addr, abi=abi["result"])


In [10]:
contract.functions.totalSupply().call()

500000000000000000000000000

In [11]:
contract.functions.name().call()

'Wilder'

In [12]:
contract.functions.symbol().call()

'WILD'